In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("anthony-wss/TL-to-CL-rate-by-validation-size")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

In [3]:
df = pd.json_normalize([summary_list[i] | config_list[i] for i in range(len(summary_list))], sep='_')
df.columns

Index(['scel', '_runtime', 'test_acc', 'train_acc', 'valid_acc',
       'training_loss', 'ure', '_step', '_timestamp', 'lr', 'algo', 'seed',
       'model', 'data_aug', 'valid_ratio', 'dataset_name',
       'data_cleaning_rate', 'best_epoch-scel_scel', 'best_epoch-scel_epoch',
       'best_epoch-scel_test_acc', 'best_epoch-scel_train_acc',
       'best_epoch-scel_valid_acc', 'best_epoch-scel_training_loss',
       'best_epoch-scel_ure', 'best_epoch-valid_acc_scel',
       'best_epoch-valid_acc_epoch', 'best_epoch-valid_acc_test_acc',
       'best_epoch-valid_acc_train_acc', 'best_epoch-valid_acc_valid_acc',
       'best_epoch-valid_acc_training_loss', 'best_epoch-valid_acc_ure',
       '_wandb_runtime', 'best_epoch-ure_test_acc', 'best_epoch-ure_train_acc',
       'best_epoch-ure_valid_acc', 'best_epoch-ure_training_loss',
       'best_epoch-ure_ure', 'best_epoch-ure_scel', 'best_epoch-ure_epoch'],
      dtype='object')

In [4]:
import pandas as pd

for seed in [0, 1]:
    print("seed", seed)
    for r in [0.2, 0.1, 0.01]:
        for algo in ['fwd-u', 'fwd-r']:
            sub_df = df.loc[(df['algo'] == algo) & (df['valid_ratio'] == r) & (df['seed'] == seed)]
            
            print(
                algo, 
                r, 
                sub_df.sort_values('valid_acc', ascending=False).iloc[0]['valid_acc'],
                sub_df.sort_values('valid_acc', ascending=False).iloc[0]['test_acc'],
                sub_df.sort_values('valid_acc', ascending=False).iloc[0]['lr'],
                sub_df.sort_values('test_acc', ascending=False).iloc[0]['test_acc'],
                sub_df.sort_values('best_epoch-ure_ure').iloc[0]['test_acc']
                # sub_df.sort_values('best_epoch-valid_acc_valid_acc', ascending=False).iloc[0]['best_epoch-valid_acc_test_acc'],
                # sub_df.sort_values('best_epoch-valid_acc_valid_acc', ascending=False).iloc[0]['lr']
            )

seed 0
fwd-u 0.2 0.347 0.3269 0.0005 0.3269 0.325
fwd-r 0.2 0.296 0.2636 0.0005 0.2838 0.1862
fwd-u 0.1 0.34 0.3374 0.0005 0.3554 0.3223
fwd-r 0.1 0.298 0.2817 1e-05 0.2817 0.197
fwd-u 0.01 0.38 0.3317 5e-05 0.3603 0.3317
fwd-r 0.01 0.3 0.2786 1e-05 0.2786 0.1896
seed 1
fwd-u 0.2 0.37 0.3649 0.0001 0.3649 0.3512
fwd-r 0.2 0.296 0.3015 1e-05 0.3015 0.2089
fwd-u 0.1 0.368 0.3525 0.0001 0.3525 0.3268
fwd-r 0.1 0.302 0.2997 1e-05 0.2997 0.2118
fwd-u 0.01 0.4 0.343 5e-05 0.3587 0.343
fwd-r 0.01 0.4 0.3078 1e-05 0.3078 0.2056
